<img src="./figs/stages.png" height=100px/>

In [1]:
import warnings

warnings.filterwarnings("ignore")

from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)

### 3. Querying

#### 3.1 Retrieve

In [2]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

In [3]:
retrieved_documents = retriever.retrieve("What's UA mission?")

In [4]:
len(retrieved_documents)

3

In [5]:
# each document has a similarity score
retrieved_documents[0].score

0.8314836327430332

In [6]:
retrieved_documents[0].text

"The 1990s marked a new phase in the UA's evolution. New priorities such as internationalisation and cooperation are redefined, namely through participation in European programmes, strengthening relations with Portuguese-speaking and Latin American countries, participating in networks and consortia of international universities and signing protocols with institutions,organisations and companies in Portugal and abroad.\n\nThe collaboration of Aveiro University with the local community and society as as whole is reinforced by the UA's engagement in promoting the transfer of knowledge, technology and innovation within the world of business. At the same time, decisive steps are being taken to promote continuous training and distance learning programmes, which were becoming increasingly important in meeting the educational needs of different publics. In the course of its growth, the University has always striven to provide students and lecturers with excellent working spaces, scientific inf

In [7]:
retrieved_documents[0].metadata

{'file_path': '/var/home/luminoso/clouds/github/dspt-handson-llamaindex/data/about_ua.txt',
 'file_name': 'about_ua.txt',
 'file_type': 'text/plain',
 'file_size': 6110,
 'creation_date': '2024-05-27',
 'last_modified_date': '2024-05-27'}

In [8]:
retrieved_documents = retriever.retrieve("What's in the food menu for friday dinner?")

In [9]:
retrieved_documents[0].text

'Cantina Santiago\nCantina com serviço de takeaway ao almoço - pode encomendar até às 11h do próprio dia\n\n \nDia \tEmenta \t\t\nAlmoço Terça 28/05/2024 \t\nSopa - Creme de ervilhas\nPrato Carne - Rojões e batata corada\nPrato Peixe - Carapau grelhado com arroz de espinafres e cenoura\nPrato Vegetariano - Bolonhesa de soja e massa integral\n\t\t\nJantar Terça 28/05/2024 \t\nSopa - Creme de ervilhas\nPrato Peixe - Massada de marisco\nprato Vegetariano - Tofu salteado com alho francês e batata\n\t\t\nAlmoço Quarta 29/05/2024 \t\nSopa - Caldo verde\nPrato Carne - Strogonoff de vitela e puré de batata\nPrato Peixe - Solha frita e arroz de tomate\nPrato Vegetariano - Paella e arroz integral\n\nJantar Quarta 29/05/2024 \t\nSopa - Caldo verde\nPrato Carne - Peito de peru com molho de mostarda e arroz\nPrato Vegetariano - Seitan agridoce e millet\n\nAlmoço Quinta 30/05/2024 \t\nENCERRADO - FERIADO\n\t\t\nJantar Quinta 30/05/2024 \t\nENCERRADO - FERIADO\n\t\t\nAlmoço Sexta 31/05/2024 \t\nSopa 

#### 3.2 Mixing sources and prompt

In [10]:
query_engine = index.as_query_engine()

In [11]:
prompts_dict = query_engine.get_prompts()
print(prompts_dict["response_synthesizer:text_qa_template"].get_template())

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


#### 3.4 Final prompt

In [12]:
import llama_index.core

llama_index.core.set_global_handler("simple")

In [13]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [14]:
response = query_engine.query("What's in the menu for friday dinner?")
response.response

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7f3ad737bf60>, 'json_data': {'input': ["What's in the menu for friday dinner?"], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7f3ad737bf60>, 'json_data': {'input': ["What's in the menu for friday dinner?"], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/embeddings
Sending HTTP Request: POST https://api.openai.com/v1/embeddings
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
send_request_headers.complete
DEBUG:httpcore.http11:send_request_b

'Sopa - Sopa de espinafres\nPrato Carne - Jardineira mista e batata\nPrato Vegetariano - Caçarola de legumes e massa integral'